# Patch Gan Discriminator Test.

In this notebook, we want to see if the patchGAN Discriminator brings something. 
To do that, we will train the wiener MTSGAN on the same dataset as before. 
we will evaluate it and see if is brings something.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from configs.Metric import Metric
from configs.SimulatedData import Proposed
from utils.metric import signature_on_batch, signature_metric
from datetime import datetime
import io

gpus = tf.config.list_physical_devices('GPU')

if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs.
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized.
    print(e)

# Load the Data.

In [ ]:
config= Proposed()

SEQUENCE_LENGTH = config.sequence_lenght_in_sample
GRANUARITY = config.granularity
OVERLAP = config.overlap
BS = config.batch_size
EPOCHS = 10 # config.epochs
NUM_SEQUENCE_TO_GENERATE = config.met_params.sequence_to_generate

TRAIN_DISCRIMINATOR_STEP = 10

SIMULATED_DATA_PATH = "data/simulated_dataset/01 - Source Domain.h5"
N_SAMPLE_WIENER = SEQUENCE_LENGTH//4
FEAT_WIENER = 2
NOISE_DIM= (N_SAMPLE_WIENER, FEAT_WIENER)

In [ ]:
def convert_dataframe_to_tensorflow_sequences(df:pd.DataFrame, sequence_lenght_in_sample, granularity, shift_between_sequences, batch_size, shuffle=True):
    sequence_lenght = int(sequence_lenght_in_sample*granularity)

    dset = tf.data.Dataset.from_tensor_slices(df.values)
    dset = dset.window(sequence_lenght , shift=shift_between_sequences, stride=granularity).flat_map(lambda x: x.batch(sequence_lenght_in_sample, drop_remainder=True))

    if shuffle:
        dset= dset.shuffle(256)

    dset = dset.batch(batch_size, drop_remainder=True)

    dset = dset.cache().prefetch(10)

    return dset

In [ ]:
df_simulated = pd.read_hdf(SIMULATED_DATA_PATH)
df_simulated = df_simulated.drop(columns="labels")

dset_simulated = convert_dataframe_to_tensorflow_sequences(
    df_simulated, 
    SEQUENCE_LENGTH, 
    GRANUARITY, 
    int(OVERLAP* SEQUENCE_LENGTH),
    BS
)

## Verify if it is correct

In [ ]:
sequence = next(iter(dset_simulated))[0]
print(sequence.shape)

plt.figure(figsize=(18, 5))
plt.title("Simulated Sequence.")
for i in range(sequence.shape[1]):
    plt.plot(sequence[:, i], label=f'feat {i+1}')
plt.grid()
plt.legend()
plt.show()

## Make some Wiener Noise.

In [ ]:
def wiener_process(batch:int, n_sample_wiener:int, n_feat_wiener:int):
    d_noise = tf.random.normal([batch, n_sample_wiener, n_feat_wiener])
    wiener_noise = tf.math.cumsum(d_noise, axis=1)
    return wiener_noise

seed = wiener_process(NUM_SEQUENCE_TO_GENERATE, N_SAMPLE_WIENER, FEAT_WIENER)

In [ ]:
def draw_arrow(A, B, color="b"):
    plt.arrow(A[0], A[1], B[0] - A[0], B[1] - A[1],
              length_includes_head=True, color=color)
    
def draw_arrows(xs, ys, color="b"):
    for i in range(xs.shape[0]-1):
        point0 = [xs[i], ys[i]]
        point1 = [xs[i+1], ys[i+1]]
        draw_arrow(point0, point1, color=color)

plt.figure(figsize=(18, 5))
plt.title("Example of the wiener process.")

draw_arrows(seed[0,:,0], seed[0,:,1], color="tab:blue")
plt.scatter(seed[0,:,0], seed[0,:,1], label='Wiener Process.', color='tab:blue')

plt.grid()
plt.legend()

## Make Model Architectures.

In [ ]:
from tensorflow.keras import layers

def make_generator(noise_shape:tuple, n_signals:int, seq_length:int):
    
    _input = tf.keras.Input(noise_shape)

    # Make a simple Wiener process projector.
    wiener_encoded= layers.Flatten()(_input)
    wiener_encoded= tf.keras.layers.Dense(tf.math.reduce_prod(noise_shape))(wiener_encoded)
    wiener_encoded= layers.BatchNormalization()(wiener_encoded)
    wiener_encoded= layers.LeakyReLU()(wiener_encoded)

    x= tf.keras.layers.Dense(n_signals*seq_length//8)(wiener_encoded)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)

    x= tf.keras.layers.Reshape((seq_length//8, n_signals))(x)

    x= tf.keras.layers.Conv1DTranspose(256, 5, 1, padding='same')(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)

    x= tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same')(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)
    
    x= tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same')(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)

    x= tf.keras.layers.Conv1DTranspose(128, 5, 2, padding='same')(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)

    x= tf.keras.layers.Conv1DTranspose(n_signals, 3, 1, padding='same')(x)
    x= layers.BatchNormalization()(x)
    x= layers.LeakyReLU()(x)

    _model = tf.keras.Model(_input, x)
    wiener_encoder = tf.keras.Model(_input, wiener_encoded)

    return _model, wiener_encoder


generator, wiener_encoder = make_generator(NOISE_DIM, sequence.shape[1], SEQUENCE_LENGTH)
generator.summary()
NOISE_DIM

In [ ]:
def make_discriminator(seq_length:int, n_feat:int):
    _input = tf.keras.Input((seq_length, n_feat))
    x = tf.keras.layers.Conv1D(128, 5, 2, padding='same')(_input)
    x = layers.LeakyReLU()(x)
    # x = layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv1D(256, 5, 2, padding='same')(x)
    x = layers.LeakyReLU()(x)
    # x = layers.Dropout(0.3)(x)

    x = tf.keras.layers.Conv1D(256, 5, 2, padding='same')(x)
    x = layers.LeakyReLU()(x)

    _output = tf.keras.layers.Conv1D(n_feat, 5, 2, padding='same', activation='sigmoid')(x)

    model = tf.keras.Model(_input, _output)
    early_predictor = tf.keras.Model(_input, x)

    return model, early_predictor

discriminator, early_predictor = make_discriminator(SEQUENCE_LENGTH, sequence.shape[1])

discriminator.summary()

### Plot a Sequence.

In [ ]:
generated = generator(seed)

# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):

    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Several Generations")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(df_simulated)

    plt.tight_layout()
    plt.show()

plot_several_generations(generated) 


## Define losses

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def similarity_loss(extracted_features:np.ndarray):
    anchor = extracted_features[0]
    return tf.exp(-(tf.norm(extracted_features[1:]- anchor)))

In [ ]:
## Generate the reference signature.

real_sequence_batch = next(iter(dset_simulated))

real_batch_signature= signature_on_batch(real_sequence_batch, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)
generated_batch_signature= signature_on_batch(generated, config.met_params.ins, config.met_params.outs, config.met_params.signature_length)

signature_metric(real_batch_signature, generated_batch_signature)

In [ ]:
generator_optimizer = tf.keras.optimizers.RMSprop(1e-2)
discriminator_optimizer = tf.keras.optimizers.RMSprop(2e-3)    

# generator_optimizer = tf.keras.optimizers.Adam(1e-4)
# discriminator_optimizer = tf.keras.optimizers.Adam(1e-6)    

## Define metrics 

In [ ]:
generator_metric = tf.keras.metrics.Mean()
discriminator_metric = tf.keras.metrics.Mean()
similarity_metric = tf.keras.metrics.Mean()
correlation_metric = tf.keras.metrics.Mean()

## Tensorboard logs.

In [ ]:
date_str = datetime.now().strftime('%Y-%m-%d_%H_%M_%S')

BASE_DIR = f"log - patchGAN Test/{date_str} - wiener 2D PatchGAN Discr"
TRAIN_LOGS_DIR_PATH = f"{BASE_DIR}/fit"
GENERATION_LOG = f"{BASE_DIR}/Generations"

TRAIN_SUMMARY_WRITER = tf.summary.create_file_writer(TRAIN_LOGS_DIR_PATH)


def plot_to_buff(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):
    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    fig = plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()

    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    plt.close(fig)
    return buf


def log_losses(epoch, plot_buf):
    image = tf.image.decode_png(plot_buf.getvalue(), channels=4)
    image = tf.expand_dims(image, 0)

    with TRAIN_SUMMARY_WRITER.as_default():
        tf.summary.scalar("Generator Loss", generator_metric.result(), step=epoch)
        tf.summary.scalar("Discriminator Loss", discriminator_metric.result(), step=epoch)
        tf.summary.scalar("Mode Colapsing ?", similarity_metric.result(), step=epoch)
        tf.summary.scalar("Correlation Metric", correlation_metric.result(), step=epoch)

        tf.summary.image("Training Generations", image, step=epoch)


# Training Functions

In [ ]:
def generate_plots(noise, save_to):
    generated = generator(noise)

    fig =plt.figure(figsize=(18, 5))
    plt.title("Generation of the GAN during Training.")
    for i in range(generated.shape[-1]):
        plt.plot(generated[0, :, i], label=f'feat {i+1}')
    plt.grid()
    plt.legend()

    plt.savefig(save_to)
    plt.close(fig)

In [ ]:
# @tf.function
def train_step(images, update_discriminator:True):
    alpha= 1  
    # noise = tf.random.normal([BS, NOISE_DIM])
    noise= wiener_process(BS, N_SAMPLE_WIENER, FEAT_WIENER)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_images = generator(noise, training=True)

      real_output = discriminator(images, training=update_discriminator)
      fake_output = discriminator(generated_images, training=update_discriminator)
      extracted_feat = early_predictor(generated_images, training=False)

      sim_loss = similarity_loss(extracted_feat)
      gen_loss = generator_loss(fake_output)+ alpha* sim_loss
      disc_loss = discriminator_loss(real_output, fake_output)

    if update_discriminator == True:
      gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
      discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))


    # Save metric for display
    generator_metric(gen_loss)
    discriminator_metric(disc_loss)
    similarity_metric(sim_loss)

In [ ]:
def train(dataset, epochs):
  losses = []
  total_steps = "?"
  for epoch in range(epochs):
    start = time.time()

    generator_metric.reset_states()
    discriminator_metric.reset_states()

    for s, image_batch in enumerate(dataset):
      update_discriminator = s % TRAIN_DISCRIMINATOR_STEP == 0



      train_step(image_batch, update_discriminator)
      print(f"\r e {epoch}/{epochs}, s {s}/{total_steps}: Gen {generator_metric.result():0.4f}; Disc {discriminator_metric.result():0.4f} sim loss: {similarity_metric.result():0.4f}", end="")

    if epoch == 0:
      total_steps = s

    stop = time.time()
    print()
    print(f"\r[+] Epoch {epoch}/{epochs} in {(stop-start):0.4f} seconds. ({(stop-start)/total_steps:0.4f} s/step)")

    generate_plots(seed, f"imgs/GAN_generations/{epoch}.png")
    # Make generations on seed
    seed_generation = generator(seed, training=False)
    buff = plot_to_buff(seed_generation)

    batch_signature = signature_on_batch(seed_generation, [0, 1], [2, 3, 4, 5], config.met_params.signature_length)
    signature_difference = signature_metric(real_batch_signature, batch_signature)
    correlation_metric(signature_difference)
    
    l = [generator_metric.result(), discriminator_metric.result()]
    losses.append(l)
    log_losses(epoch, buff)

  return np.array(losses)


In [19]:
training_losses = train(dset_simulated, EPOCHS)

KeyboardInterrupt: 

In [ ]:
generator.save(f"{BASE_DIR}/generator.h5")

## Plot Losses.

In [ ]:
plt.figure(figsize=(18, 5))
plt.title("Training Losses.")
plt.plot(training_losses[:, 0], ".-", label="Generator Loss")
plt.plot(training_losses[:, 1], ".-", label="Discriminator Loss")
plt.grid()
plt.legend()
plt.show()

# Let's Test it!

In [ ]:
after_training_generations = generator(seed, training=False)

plt.figure(figsize=(18, 5))
plt.title("Generation of the GAN whitout Training.")
for i in range(after_training_generations.shape[-1]):
    plt.plot(after_training_generations[0, :, i], label=f'feat {i+1}')
plt.grid()
plt.legend()
plt.show()

In [ ]:
after_training_decision = discriminator(after_training_generations)
after_training_decision[0]

## Test if Mode Colapsing

In [ ]:
# after_training_generations
def plot_several_generations(generations:np.ndarray, nvertical:int=3, nhoriz:int=3):

    legend = [f"feat {j}" for j in range(generations.shape[-1])]

    plt.figure(figsize=(18, 10))
    plt.suptitle("Generations After GAN Training.")

    for i in range(nvertical* nhoriz):
        ax = plt.subplot(nvertical, nhoriz, i+ 1)
        ax.set_title(f"sequence {i+1}")

        plt.plot(generations[i])
        ax.grid(True)
        plt.legend(legend)

    plt.tight_layout()
    plt.show()

plot_several_generations(after_training_generations)